In [72]:
import pandas as pd
import os, sys
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
  sys.path.append(src_path)

from load_data import load_data

users, games, ratings = load_data()
ratings.head(1)



,username,bgg_id,game_name,rating,owned,num_plays,collection_date,weight,avg_rating,num_ratings,categories,mechanics
0,AlexCast,267127,Aerion,7.0,1,1,2025-09-18T17:17:05.721113,1.9677,7.28969,1974.0,"[Aviation / Flight, Card Game, Dice, Fantasy]","[Cooperative Game, Dice Rolling, Open Drafting..."


In [73]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF

ratings_test = ratings.copy()

ratings_df = ratings_test.pivot_table(
    index='game_name',
    columns='username',
    values='rating',
    fill_value=0 
)

original_df = ratings_test.pivot_table(
    index='game_name',
    columns='username',
    values='rating'
)

ratings_df.head(1)

username,-toni-,0010,007King_Kong,015599m,020907,0492372665,0ddjob,0time,1000rpm,105JS2687S2003,...,urbanlabyrinth,user92,wbgfun,wigunowner,wmspringer,wwalker98,xatsmann,xenotime,xxravenxx,yijinim
game_name,,,,,,,,,,,,,,,,,,,,,
"""Not Another One!""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
def _rank_calculation(data: pd.DataFrame):
    benchmark = np.linalg.norm(data, ord='fro') * 0.0001
    rank = 3

    while True:
        model = NMF(n_components=rank, init='random', random_state=0, max_iter=500)
        W = model.fit_transform(data)
        H = model.components_
        V = W @ H
        
        RMSE = np.sqrt(mean_squared_error(data, V))
        if RMSE < benchmark:
            return rank, V
        print(f"Rank {rank}: RMSE = {RMSE:.4f}")

        rank += 1

#_rank_calculation(ratings_df)
optimal_rank = 12

In [75]:
model = NMF(n_components=optimal_rank)
model.fit(ratings_df)

H = pd.DataFrame(model.components_)    
W = pd.DataFrame(model.transform(ratings_df))    
predicted_df = pd.DataFrame(np.dot(W,H), columns=ratings_df.columns)
predicted_df.index = ratings_df.index
predicted_df

username,-toni-,0010,007King_Kong,015599m,020907,0492372665,0ddjob,0time,1000rpm,105JS2687S2003,...,urbanlabyrinth,user92,wbgfun,wigunowner,wmspringer,wwalker98,xatsmann,xenotime,xxravenxx,yijinim
game_name,,,,,,,,,,,,,,,,,,,,,
"""Not Another One!""",0.027935,0.000000,0.010270,0.000000,0.000000,0.002247,0.024727,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.014217,0.000000,0.000000,0.009191,0.031051,0.000000,0.000000
"""Oh My God! There's An Axe In My Head."" The Game of International Diplomacy",0.075624,0.000749,0.006099,0.020004,0.000095,0.003717,0.113948,0.000000,0.305647,0.015845,...,0.000000,0.004240,0.109552,0.006757,0.108033,0.000000,0.004575,0.036865,0.005927,0.000000
'65: Squad-Level Combat in the Jungles of Vietnam,0.000851,0.001826,0.000226,0.000230,0.000000,0.019890,0.012900,0.001658,0.003182,0.000000,...,0.001734,0.000275,0.000000,0.000349,0.006724,0.007279,0.000149,0.005827,0.000333,0.000260
'ne Tüte Chips,0.017182,0.072401,0.000126,0.000000,0.001288,0.000028,0.000303,0.002873,0.022900,0.000000,...,0.000000,0.016236,0.025451,0.001992,0.000000,0.049185,0.000113,0.000381,0.007677,0.003494
(May Cause) Side Effects,0.000000,0.001242,0.000000,0.001043,0.000000,0.006166,0.002816,0.000000,0.047472,0.026287,...,0.000000,0.000000,0.166547,0.000000,0.000000,0.000000,0.000000,0.002972,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황금 앙크,0.003819,0.015593,0.000135,0.000126,0.000256,0.011841,0.007680,0.001559,0.006305,0.000000,...,0.001112,0.003396,0.005224,0.000559,0.003992,0.014433,0.000089,0.003469,0.002933,0.000952
황금의 땅 엘도라도,0.018915,0.000000,0.001504,0.004783,0.000125,0.000027,0.025737,0.000000,0.068191,0.000000,...,0.000350,0.000961,0.002185,0.002098,0.024899,0.001343,0.001456,0.008318,0.002683,0.000441
황혼의 투쟁,0.033662,0.080061,0.001170,0.003914,0.001505,0.012737,0.025552,0.004187,0.080405,0.000000,...,0.001583,0.018488,0.029385,0.004201,0.021588,0.059994,0.001146,0.009355,0.012182,0.005029


In [76]:
def user_recommendations(user):
    user_preds = predicted_df[user].sort_values(ascending=False)

    top_10_games = {}
    for game in user_preds.index:
        if pd.isna(original_df.at[game, user]):
            top_10_games[game] = user_preds[game]
            print(f"{game:<40} {user_preds[game]:.4f}")
        if len(top_10_games) == 10:
            break

    return top_10_games

test_user = user_recommendations("AlexCast")


7 Wonders Duel                           2.2328
Codenames                                2.0593
The Castles of Burgundy                  1.9819
7 Wonders                                1.9814
Carcassonne                              1.9564
Ark Nova                                 1.9329
Lost Ruins of Arnak                      1.9007
Spirit Island                            1.8706
The Crew: The Quest for Planet Nine      1.8036
Brass: Birmingham                        1.7646
